In [19]:
import macrobond_data_api as mda
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from jinja2 import Environment, FileSystemLoader

In [20]:
colors = ['#1B365D', '#4964A2', '#256BA2', '#8E51A8', '#58A291', '#CD545B', '#004F59', '#F68D2E', '#008578', '#934054', '#25282A', '#333F48', '#676d72', '#C7C9C7']

In [21]:
universe = ['gbcpi', #inflation
            'gbcons3148', #rental index
            'gbcons6501'] #house price index

dictionary = {'gbcpi': 'CPI',
              'gbcons3148': 'Rental Index',
              'gbcons6501': 'House Price Index'}


In [22]:
entities = mda.get_series(universe)
df = pd.json_normalize([x.to_dict() for x in entities])
df['Name'] = df['Name'].map(lambda x: dictionary[x])

In [23]:
df_ts = df.set_index(['Name'])[['Dates','Values']].apply(pd.Series.explode).pivot_table(index='Dates', columns='Name', values='Values')

In [24]:
def lineplots(Df, Title=None):
    fig = go.Figure()
    for col in Df.columns:
        fig.add_trace(go.Scatter(x=Df.index, y=Df[col], name=col, line=dict(color=colors[Df.columns.get_loc(col)])))

    fig.update_layout(
        autosize=False,
        width=1000,
        height=600,
        xaxis_title="Date",
        yaxis_title="Index",
        font=dict(
            family="Montserrat",
            size=12,
            color="#7f7f7f"
        )
    )

    fig.add_annotation(text="Source: Macrobond, ONS, GOV.UK",
                    xref="paper", yref="paper",
                    x=1, y=-0.1, showarrow=False)
    
    if Title != None:
        fig.update_layout(title=Title)

    fig.show()
    return fig

In [25]:
fig_1 = lineplots(df_ts)

In [26]:
# remove the CPI's influence on the other two series

df_ts_cpi_removed = df_ts.copy()
df_ts_cpi_removed['House Price Index'] = df_ts_cpi_removed['House Price Index'] - df_ts_cpi_removed['CPI']
df_ts_cpi_removed['Rental Index'] = df_ts_cpi_removed['Rental Index'] - df_ts_cpi_removed['CPI']
df_ts_cpi_removed = df_ts_cpi_removed.drop(columns=['CPI'])


In [27]:
fig_2 = lineplots(df_ts_cpi_removed)

In [28]:
# Set up the Jinja2 environment and load the template
env = Environment(loader=FileSystemLoader('.'))
template = env.get_template('templates.html')

# Render the template with the desired data
result = template.render(
                         chart_1 = fig_1.to_html(full_html=True, include_plotlyjs='cdn', default_height=800, default_width=1500),
                         chart_2 = fig_2.to_html(full_html=True, include_plotlyjs='cdn', default_height=800, default_width=1500))

# Save the rendered result to an HTML file
with open('london_house_index.html', 'w', encoding='utf-8-sig') as f:
    f.write(result)